### Imports

In [6]:
import pandas as pd
import surprise
from surprise.model_selection import train_test_split
from surprise.prediction_algorithms import SVD
from surprise.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from surprise import accuracy
import pickle
from surprise.prediction_algorithms import knns
from surprise.similarities import cosine, msd, pearson
from surprise import accuracy

In [7]:
links = pd.read_csv('/Users/gabrielwarner/Downloads/ml-latest-small/links.csv')

In [8]:
movies = pd.read_csv('/Users/gabrielwarner/Downloads/ml-latest-small/movies.csv')

In [9]:
ratings = pd.read_csv('/Users/gabrielwarner/Downloads/ml-latest-small/ratings.csv')

In [10]:
tags = pd.read_csv('/Users/gabrielwarner/Downloads/ml-latest-small/tags.csv')

In [11]:
ratings = ratings.drop('timestamp', axis=1)

### Merge Data Frames

In [12]:
new = pd.merge(links, movies, on='movieId')

In [13]:
new_2 = pd.merge(new, ratings, on='movieId')

In [14]:
new_3 = pd.merge(new_2, tags, on='movieId')

### Preprocessing

In [15]:
# replace | with , in the genres column
new_3['genres'] = new_3['genres'].str.replace('|',' , ')

In [ ]:
# convert df to a csv file
new_3.to_csv('/Users/gabrielwarner/Data-Science/Projects/phase_4/movies_2.csv')

### Surprise Preprocessing

In [16]:
reader = surprise.Reader(rating_scale = (4., 5.))

In [17]:
data = surprise.Dataset.load_from_df(ratings, reader)

In [18]:
trainset, testset = train_test_split(data, test_size=0.2)

### KNNS Basic model

In [ ]:
# Initiate knn model and fir to trainset
basic = knns.KNNBasic()
basic.fit(trainset)

In [ ]:
# make predictions wih the model
predictions = basic.test(testset)

In [ ]:
# Get the RMSE score
print(accuracy.rmse(predictions))

### KNN means model

In [ ]:
sim_pearson = {'name':'pearson', 'user_based':False}
knn_means = knns.KNNWithMeans(sim_options=sim_pearson)
knn_means.fit(trainset)
predictions = knn_means.test(testset)
print(accuracy.rmse(predictions))

### Suprise with grid search SVD

In [11]:
param_grid = {'n_factors':[20, 50, 75, 100],'n_epochs': [5, 6, 7, 8, 9, 10, 11], 'lr_all': [0.002, .003, .004, 0.005],
               'reg_all': [.02, 0.4, .5, 0.6]}
gs = GridSearchCV(SVD,param_grid=param_grid,n_jobs = -1,joblib_verbose=5)
gs.fit(data)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    4.5s
[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed:   29.9s
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:  1.8min
[Parallel(n_jobs=-1)]: Done 272 tasks      | elapsed:  3.0min
[Parallel(n_jobs=-1)]: Done 434 tasks      | elapsed:  4.4min
[Parallel(n_jobs=-1)]: Done 632 tasks      | elapsed:  5.9min
[Parallel(n_jobs=-1)]: Done 866 tasks      | elapsed:  8.4min
[Parallel(n_jobs=-1)]: Done 1136 tasks      | elapsed: 12.2min
[Parallel(n_jobs=-1)]: Done 1442 tasks      | elapsed: 16.4min
[Parallel(n_jobs=-1)]: Done 1784 tasks      | elapsed: 22.0min
[Parallel(n_jobs=-1)]: Done 2162 tasks      | elapsed: 28.9min
[Parallel(n_jobs=-1)]: Done 2240 out of 2240 | elapsed: 30.6min finished


In [14]:
svd = SVD(n_factors=100, n_epochs=10, lr_all=0.005, reg_all=0.4)
svd.fit(trainset)
predictions = svd.test(testset)
print(accuracy.rmse(predictions))

RMSE: 1.1547
1.1546672240307976


### Pickle

In [33]:
# Save the trained model as a pickle string.
saved_model = pickle.dumps(gs)
 
# Load the pickled model
gs_from_pickle = pickle.loads(saved_model)
 
# Use the loaded pickled model to make predictions
#gs_from_pickle.predict(trainset)

### Making Predictions

In [4]:
import pandas as pd

In [23]:
ratings_matrix = merged_data.pivot_table(index='userId',columns='title',values='rating')
ratings_matrix.to_csv('./ratings_matrix.csv', index = False)

In [24]:
def movie_recommendations(movie_name):
    user_ratings = ratings_matrix[movie_name]
    correlation_with_movie = pd.DataFrame(ratings_matrix.corrwith(user_ratings))
    correlation_with_movie = correlation_with_movie.join(rating_data['# of ratings'])
    correlation_with_movie.columns = [f'Corr. With {movie_name}', '# of Ratings']
    correlation_with_movie.index.names = ['Movie Title']
    #return correlation_with_movie[correlation_with_movie['# of Ratings'] > 50].sort_values(f'Corr. With {movie_name}', ascending = False).iloc[1:,:].head(5)
    return correlation_with_movie[correlation_with_movie['# of Ratings'] > 50].sort_values(f'Corr. With {movie_name}', ascending = False).iloc[1:,:0].head(5)

merged_data = pd.merge(ratings , movies, on='movieId')

rating_data = pd.DataFrame(merged_data.groupby('title')['rating','movieId','userId'].mean())
rating_data['# of ratings'] = merged_data.groupby('title')['rating'].count()





<ipython-input-24-9cb3efdb38b8>:12: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  rating_data = pd.DataFrame(merged_data.groupby('title')['rating','movieId','userId'].mean())


In [25]:
movie_recommendations('Toy Story (1995)')

/Users/gabrielwarner/opt/anaconda3/envs/learn-env/lib/python3.8/site-packages/numpy/lib/function_base.py:2526: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar)
/Users/gabrielwarner/opt/anaconda3/envs/learn-env/lib/python3.8/site-packages/numpy/lib/function_base.py:2455: RuntimeWarning: divide by zero encountered in true_divide
  c *= np.true_divide(1, fact)


""
Movie Title
Toy Story 2 (1999)
Arachnophobia (1990)
"Incredibles, The (2004)"
Finding Nemo (2003)
Aladdin (1992)
